# Проект для "Викишоп"

## Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. 

Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

В распоряжении набор данных с разметкой о токсичности правок.

**Условие заказчика**: метрика качества `F1` не меньше **0,75**

## Содержание

1. Знакомство с данными
2. Подготовка данных
3. Обучение разных моделей
4. Общий вывод

## Знакомство с данными

Для начала посмотрим на предоставленные данные заказчиком: набор данных с разметкой о токсичности правок.

In [1]:
try:
    !pip install catboost
except:
    %pip install catboost

In [2]:
# имопрт библиотек для работы с данными
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
# импорт библиотек для работы с текстом
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('punkt');
nltk.download('wordnet');
nltk.download('omw-1.4');
nltk.download('stopwords');
nltk.download('averaged_perceptron_tagger');
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alexander\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alexander\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Alexander\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alexander\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Alexander\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

In [5]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [6]:
display(data.head())
data.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Имеем практически 160 тысяч комментариев, объем датасета составляет 2.5 МБ. Видим, что таблица имеет два столбца:

* `text` - текст комментария;
* `toxic` - разметка токсичности комментария
    * **0** - комментарий нетоксичен;
    * **1** - комментарий токсичен.

Для решения задачи будем прогнозировать целевой признак `toxic`. Для этого посмотрим на соотношение классов в выборке.

**Обработка пропусков**

Посмотрим на количество пропусков в наших данных:

In [7]:
data.isna().mean()

text     0.0
toxic    0.0
dtype: float64

На основе общей информации о датасете и проверки на наличие пропусков, можем скзазать, что пропусков нет.

Теперь посмотрим на наличие дубликатов:

**Обработка дубликатов**

In [8]:
data.duplicated().sum()

0

Дубликатов в данных нет. Теперь посмотрим на соотношение классов в выборке:

In [9]:
data.toxic.value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Видим, что классы в предоставленных данных несбалансированы. При проектировании моделей прогнозирования необходимо учесть эту проблему.

### Вывод

В процессе знакомства с данными обнаружили следующие проблемы:

1. В текстах комментариев имеются лишние символы. Необходимо провести фильтрацию лишних значений;
2. Необходимо для обучения сформировать набор комментариев в лемматизированном виде;
3. Учесть сильный дисбаланс классов при обучении модели.

На данный момент, данные не готовы для предобработки. Необходимо провести подготовку данных.

## Подготовка данных

Для начала важно учесть следующие проблемы:

1. Какие модели будем использовать для обучения?
2. Как будем проводить фильтрацию?
3. Как будем представлять наши данные в векотрном виде? 

Для решения задачи заказчика предлагается использовать следующие модели:

* Подбор моделей:
    1. `LinearRegression()`;
    2. `RandomForestClassifier()`;
    3. `CatBoostClassifier()`;
    4. `LGBMCLassifier()`;

* Фильтрацию будем проводить следующим образом:
    1. Фильтрация лишних значений;
    2. Реализация лемматизации для отфильтрованного от лишних символов в тексте. 

* Отфильтрованные данные будем представлять в следующем виде:
    1. Мешок слов;
    2. `TF-IDF`; 

Приступим к выполнению задания.

Для начала реализуем очистку от лишних символов. Данная процедура будет реализовываться при инициализации объекта класса. Так как для корректной работы необходимо избавляться от некорректных симоволов. Поэтому приняли решение очищать от символов всегда.

In [10]:
'''
    Скрипт-класс для прогнозирования класса токсичности текста
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    nlp_prepoc - отцовский класс инициализизатор. 
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    С его помощью выполняется вся необходимая предобработка текста:
        . Очисктка от лишних символов
        . Лемматазция
        . Формирование корпуса текста
        . Разбиение на выборки: тестовая и обучающая
        . После инициализации удаляются лишние для обучения данные
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    models - класс для обучения моделей
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    Наследует все атрибуты от отцовского класса
    В данном классе реализованы следующие методы:
    . take_params - возвращает набор параметров для указанной модели обучения
        ARGS: model_type - имя модели

    . pipeline - возвращает сформированный пайплайн
        ARGS: model - указанная модель
              vector - указанный векторизатор

    . take_res - возвращает результат обучения пайплайна (точнее лучшую модель для обучения). 
                 используется грубый перебор и трехкратная кросс-валидация
        ARGS: model - указанная модель
              vector - указанный векторизатор
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    КАК ИНИЦИАЛИЗИРОВАТЬ КЛАСС?
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   
    . передать датафрейм
    . указать фичи (текст)
    . указать ЦП (таргет)
    . нужна ли лемматизация? (True/False) 
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Для обучения:
    . определить инициализатор вне класса;
    . определить модель вне класса.
ПЕРЕДАВАТЬ ТАК: obj_name.take_res(model, vector)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
'''


class nlp_preproc:
    def __init__(self, data: pd.DataFrame, 
    target: str, 
    text_features: str,
    lemma = None):

        self.data = data.copy()
        self.data['text filter'] = self.data[text_features].apply(self.clear_value)
        
        if lemma:
            self.data['lemm_text'] = self.data['text filter'].apply(self.lemmatize)
            print('Your features are lemmatized!')
            self.feature = self.data['lemm_text']
        else:
            print('Your features are not lemmatized! Initialize param "lemm" for lemmatize')
            self.feature = self.data['text filter']

        self.target = self.data[target]

        self.X_train, self.X_test, self.y_train, self.y_test =  (
            train_test_split(self.feature, self.target, test_size=.2, random_state=54321, stratify=self.target))
        
## ПРАВКИ ПО ЗАМЕЧНИЮ: ДОБАВЛЕН УЧЕТ POS-тега

    def get_wordnetpos(self, text):
        tag = nltk.pos_tag([text])[0][1][0]
        tag_dict = {'J': wordnet.ADJ,
                   'N': wordnet.NOUN,
                   'V': wordnet.VERB,
                   'R': wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)
    
    def lemmatize(self, text):
        text = text.lower()
        m = WordNetLemmatizer()
        word_txt = word_tokenize(text)
        return ' '.join([m.lemmatize(txt, self.get_wordnetpos(txt)) for txt in word_txt])
         
    def clear_value(self, text):
        return ' '.join(re.sub(r'[^a-zA-Z]', ' ', text).split())

class models(nlp_preproc):

    def __init__(self, *args, **kwargs):
        nlp_preproc.__init__(self, *args, **kwargs)
        del self.feature
        del self.data
        del self.target
        self.test_list = []
        self.train_list = []
        self.vector_list = []
        self.model_index = []

    @staticmethod
    def take_params(model_type: str):
        if 'LogisticRegression' in model_type:
            params = {
                'mod__random_state' : [54321],
                'mod__class_weight' : ['balanced'],
                'mod__C': [15],
                'mod__solver' : ['liblinear'],
                'mod__max_iter': [1000]
            }
        elif 'RandomForest' in model_type:
            params = {
                'mod__random_state' : [54321],
                'mod__class_weight' : ['balanced'],
                'mod__n_estimators' : [51,56,5],
                'mod__criterion' : ['gini', 'entropy']
            }
        elif 'CatBoost' in model_type:
            params = {
                'mod__iterations' : [100],
                'mod__eval_metric' : ['TotalF1'],
                'mod__learning_rate' : [1],
                'mod__random_seed' : [54321],
                'mod__depth' : [120],
                'mod__thread_count' : [-1],
                'mod__bootstrap_type' : ['MVS'],
                'mod__grow_policy' : ['Lossguide']
            } 
        elif 'LGBM' in model_type:
            params = {
                'mod__n_estimators': [200],
                'mod__num_leaves': [1000,1300,250],
                'mod__max_depth' : [50],
                'mod__learning_rate' : [0.2],
                'mod__class_weight': ['balanced'],
                'mod__random_state':[54321],
                'mod__n_jobs' : [3]
            }           
        return params

    @staticmethod
    def pipeline(self, model, vector):
        pipe = Pipeline([('vec', vector),
        ('mod', model)])
        return pipe

    @staticmethod
    def grid_search(self, model, vector):
        self.grid = GridSearchCV(self.pipeline(self, model, vector), 
        self.take_params(model.__class__.__name__),
         cv = 3, 
         scoring='f1',
         n_jobs=3,
         verbose = 15)

        self.grid.fit(self.X_train, self.y_train)
        print(f'The best F1-score of {model.__class__.__name__} : {self.grid.best_score_:.3f}')

        self.model_index.append(model.__class__.__name__)
        self.train_list.append(self.grid.best_score_)
        self.vector_list.append(vector.__class__.__name__)
        return self.grid.best_estimator_;

    def take_res(self, model, vector):
        model = self.grid_search(self, model, vector)
        model.fit(self.X_train, self.y_train)
        prediction = model.predict(self.X_test)
        print(f'The final F1-score for test with {model.__class__.__name__}: {f1_score(self.y_test, prediction):.3f}')
        self.test_list.append(f1_score(self.y_test, prediction))
        del model

    def take_report(self):
        df = pd.DataFrame({
            'vectorizer': self.vector_list,
            'train results': self.train_list,
            'test results': self.test_list}, index = self.model_index)
        return df


In [11]:
res = nlp_preproc(data, 
target='toxic', 
text_features='text')

Your features are not lemmatized! Initialize param "lemm" for lemmatize


Посмотрим на полученные результат обработки текста:

In [12]:
res.feature[:5]

0    Explanation Why the edits made under my userna...
1    D aww He matches this background colour I m se...
2    Hey man I m really not trying to edit war It s...
3    More I can t make any real suggestions on impr...
4    You sir are my hero Any chance you remember wh...
Name: text filter, dtype: object

Видим, что от лишних символов мы избавились. Теперь проведем лемматизациию нашего отфильтрованного столбца.

In [13]:
res.data['text filter'].apply(res.lemmatize).head()

0    explanation why the edits make under my userna...
1    d aww he match this background colour i m seem...
2    hey man i m really not try to edit war it s ju...
3    more i can t make any real suggestion on impro...
4    you sir be my hero any chance you remember wha...
Name: text filter, dtype: object

Видим, что все комментарии приведены к лемматизированному виду, теперь можно приступить к обучению моделей.

In [14]:
del res

### Вывод

В процессе предобработки данных, устранили следующие проблемы:

1. Удалили ненужные символы;
2. Провели лемматизацию данных.

После предобработки данных приступим к обучению моделей.

## Обучение моделей

Перед обучением моделей необходимо выбрать в каком векторном представлении будут наши данные?

Для этого попробуем представить в трех вариантах:

1. Мешок слов;
2. `TF-IDF`;

Данную процедуру будем выполнять внутри класса.

In [15]:
stoplist = set(stopwords.words('english'))

Инициализируем векторизаторы:

In [16]:
# используем float для работы с LGBM
count_vector = CountVectorizer(stop_words=stoplist, dtype=np.float32)
tf_vector = TfidfVectorizer(stop_words=stoplist, dtype=np.float32)

In [17]:
test = models(data, 
target='toxic', 
text_features='text',
lemma = True)

Your features are lemmatized!


**Мешок слов**

**Логистическая регрессия**

In [18]:
test.take_res(LogisticRegression(), count_vector);

Fitting 3 folds for each of 1 candidates, totalling 3 fits
The best F1-score of LogisticRegression : 0.748
The final F1-score for test with Pipeline: 0.752


**Случайный лес**

In [19]:
test.take_res(RandomForestClassifier(), count_vector);

Fitting 3 folds for each of 6 candidates, totalling 18 fits
The best F1-score of RandomForestClassifier : 0.633
The final F1-score for test with Pipeline: 0.646


**CatBoostClassifier**

In [20]:
test.take_res(CatBoostClassifier(), count_vector);

Fitting 3 folds for each of 1 candidates, totalling 3 fits
0:	learn: 0.9060093	total: 2.88s	remaining: 4m 45s
1:	learn: 0.9365670	total: 8.55s	remaining: 6m 59s
2:	learn: 0.9429951	total: 13.8s	remaining: 7m 26s
3:	learn: 0.9461877	total: 19.5s	remaining: 7m 47s
4:	learn: 0.9486988	total: 25s	remaining: 7m 55s
5:	learn: 0.9499304	total: 31s	remaining: 8m 5s
6:	learn: 0.9510358	total: 36.8s	remaining: 8m 9s
7:	learn: 0.9518343	total: 42.7s	remaining: 8m 10s
8:	learn: 0.9526022	total: 48.5s	remaining: 8m 10s
9:	learn: 0.9531764	total: 54.6s	remaining: 8m 11s
10:	learn: 0.9539571	total: 1m	remaining: 8m 10s
11:	learn: 0.9544402	total: 1m 6s	remaining: 8m 8s
12:	learn: 0.9550403	total: 1m 12s	remaining: 8m 5s
13:	learn: 0.9555056	total: 1m 18s	remaining: 8m
14:	learn: 0.9562447	total: 1m 24s	remaining: 7m 58s
15:	learn: 0.9565560	total: 1m 30s	remaining: 7m 54s
16:	learn: 0.9568980	total: 1m 36s	remaining: 7m 50s
17:	learn: 0.9574475	total: 1m 42s	remaining: 7m 46s
18:	learn: 0.9575745	tot

**LGBMClassifier**

In [21]:
test.take_res(LGBMClassifier(), count_vector);

Fitting 3 folds for each of 3 candidates, totalling 9 fits
The best F1-score of LGBMClassifier : 0.773
The final F1-score for test with Pipeline: 0.780


**TF-IDF**

**Логистическая регрессия**

In [22]:
test.take_res(LogisticRegression(), tf_vector);

Fitting 3 folds for each of 1 candidates, totalling 3 fits
The best F1-score of LogisticRegression : 0.756
The final F1-score for test with Pipeline: 0.756


**Случайный лес**

In [23]:
test.take_res(RandomForestClassifier(), tf_vector);

Fitting 3 folds for each of 6 candidates, totalling 18 fits
The best F1-score of RandomForestClassifier : 0.624
The final F1-score for test with Pipeline: 0.645


**CatBoostClassfier**

In [24]:
test.take_res(CatBoostClassifier(), tf_vector);

Fitting 3 folds for each of 1 candidates, totalling 3 fits
0:	learn: 0.9084268	total: 5.16s	remaining: 8m 30s
1:	learn: 0.9366817	total: 13.2s	remaining: 10m 48s
2:	learn: 0.9419520	total: 21.1s	remaining: 11m 22s
3:	learn: 0.9467196	total: 29.4s	remaining: 11m 46s
4:	learn: 0.9485391	total: 37.3s	remaining: 11m 49s
5:	learn: 0.9506695	total: 45.6s	remaining: 11m 53s
6:	learn: 0.9517366	total: 54s	remaining: 11m 57s
7:	learn: 0.9527384	total: 1m 2s	remaining: 11m 57s
8:	learn: 0.9539003	total: 1m 10s	remaining: 11m 55s
9:	learn: 0.9546736	total: 1m 19s	remaining: 11m 52s
10:	learn: 0.9557262	total: 1m 27s	remaining: 11m 49s
11:	learn: 0.9565375	total: 1m 36s	remaining: 11m 44s
12:	learn: 0.9572021	total: 1m 44s	remaining: 11m 39s
13:	learn: 0.9578541	total: 1m 52s	remaining: 11m 33s
14:	learn: 0.9585230	total: 2m 1s	remaining: 11m 27s
15:	learn: 0.9589719	total: 2m 9s	remaining: 11m 20s
16:	learn: 0.9594458	total: 2m 18s	remaining: 11m 14s
17:	learn: 0.9598498	total: 2m 26s	remaining: 

**LGBMClassifier**

In [25]:
test.take_res(LGBMClassifier(), tf_vector);

Fitting 3 folds for each of 3 candidates, totalling 9 fits
The best F1-score of LGBMClassifier : 0.763
The final F1-score for test with Pipeline: 0.777


Посмотрим на полученные результаты:

In [26]:
test.take_report()

,vectorizer,train results,test results
LogisticRegression,CountVectorizer,0.748298,0.752237
RandomForestClassifier,CountVectorizer,0.633473,0.646032
CatBoostClassifier,CountVectorizer,0.742558,0.746676
LGBMClassifier,CountVectorizer,0.772583,0.779808
LogisticRegression,TfidfVectorizer,0.755907,0.755568
RandomForestClassifier,TfidfVectorizer,0.623838,0.645135
CatBoostClassifier,TfidfVectorizer,0.750249,0.765029
LGBMClassifier,TfidfVectorizer,0.763492,0.777424


### Вывод

В результате обучения моделей видим следующее:

1. Лучшей моделью в плане метрики `F1` оказалась модель `LGBMClassifier` с использованием ящика слов;
2. Худшей моделшью оказалась модель `RandomForestClassifier`с использованием ящика в слов;
3. Независимо от векторизатора с условием заказчика не справилась модель `RandomForestClassifier`.

## Общий вывод

Была дана задача:

> Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Для выполнения работы заказчиком был предоставлен набор данных с разметкой о токсичности правок.

**Условие заказчика**: метрика качества `F1` не меньше **0,75**

После знакомства с данными установили следующие проблемы:

1. В текстах комментариев имеются лишние символы. Необходимо провести фильтрацию лишних значений;
2. Необходимо для обучения сформировать набор комментариев в лемматизированном виде;
3. Учесть сильный дисбаланс классов при обучении модели.

На данный момент, данные не готовы для предобработки. Необходимо провести подготовку данных.

В процессе подготовки моделей и их обучения выявленные проблемы были учетны и исправлены.

В результате обучения получили следующие выводы:

1. Лучшей моделью в плане метрики `F1` оказалась модель `LGBMClassifier` с использованием ящика слов;
2. Худшей моделшью оказалась модель `RandomForestClassifier`с использованием ящика в слов;
3. Независимо от векторизатора с условием заказчика не справилась модель `RandomForestClassifier`.

В зависимости от того, как мы подготавливаем данные (лемматизируем ли мы текст, формируем токены, какие векторизаторы используем), а также какие гиперпараметры мы регулируем, мы можем получить разные результаты. 